---
title : "Fragility Curve uit een cache"
execute:
    output: asis
---

Voor het vervangen van een fragility curve door een eerder berekende curve maken we gebruik van een `cache`.<br>
Dit is niks anders dat een set aan voorberekende curves. <br>
De logica om te bepalen welke curve uit de cache moet worden gebruikt, moet in de python code worden geïmplementeerd.<br>
De `FragilityCurve` class biedt hiervoor de methode `load_effect_from_data_adapter`.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
from toolbox_continu_inzicht.base.data_adapter import Config, DataAdapter

path = Path.cwd() / "data_sets" / "6.fragility_curve_from_cache"
config = Config(config_path=path / "test_fragility_curve_from_cache.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

De volgende configuratie wordt gebruikt voor het maken van vervangen van een curve met een cache
```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    fragility_curve_initial:
        type: csv
        file: "fragility_curve_initial.csv"
    fragility_curve_cache_1:
        type: csv
        file: "fragility_curve_cache_1.csv"
    fragility_curve_cache_2:
        type: csv
        file: "fragility_curve_cache_2.csv"
    fragility_curve_cache_3:
        type: csv
        file: "fragility_curve_cache_3.csv"
    fragility_curve_cache_4:
        type: csv
        file: "fragility_curve_cache_4.csv"
    measures_to_cache:
        type: csv
        file: "measures_to_cache.csv"
```

In dit geval is er een initiële fragility curve en vier fragility curves in de cache.<br>

In [ ]:
from toolbox_continu_inzicht.base.fragility_curve import FragilityCurve

In [ ]:
fragility_curve = FragilityCurve(data_adapter)
fragility_curve.load("fragility_curve_initial")
fragility_curve.as_dataframe()

Hier kan bijvoorbeeld met een index worden aangegeven welke curve uit de cache moet worden gebruikt.<br>

In [ ]:
fragility_curve.cached_fragility_curves = {
    0: "fragility_curve_cache_1",
    1: "fragility_curve_cache_2",
    2: "fragility_curve_cache_3",
    3: "fragility_curve_cache_4",
}

In [ ]:
fig, ax = plt.subplots()
fragility_curve.as_dataframe().set_index("hydraulicload").plot(ax=ax)
ax.set_ylabel("Failure probability")
for i in fragility_curve.cached_fragility_curves:
    cached_curve = fragility_curve.copy()
    cached_curve.load_effect_from_data_adapter(i)
    cached_curve.as_dataframe().set_index("hydraulicload").plot(
        ax=ax, label=f"Cache {i}", alpha=0.15
    )
ax.legend(["initial curve", "Cache 1", "Cache 2", "Cache 3", "Cache 4", "Cache 5"])
ax.set_xlim(13, 16)

### Door met voorbeeld uit  _> maak een functie 

In [ ]:
curve = FragilityCurve(data_adapter)

In [ ]:
curve.load("fragility_curve_initial")

In [ ]:
curve.as_dataframe()

In [ ]:
curve.cached_fragility_curves = {
    0: "fragility_curve_cache_1",
    1: "fragility_curve_cache_2",
    2: "fragility_curve_cache_3",
    3: "fragility_curve_cache_4",
}

In [ ]:
curve.load_effect_from_data_adapter(1)

In [ ]:
curve.as_dataframe()